In [ ]:
!pip install -qU datasets

In [ ]:
from datasets import load_dataset
import torch

In [ ]:
dataset = load_dataset("claudios/code_search_net", "all", split="train")

# Lowering test dataset size to train faster
# datasetDictionary = dataset.train_test_split(test_size=0.004)

In [ ]:
print(dataset)

In [ ]:
sample = dataset[121212]
sampleName = sample['func_name']
sampleCode = sample['whole_func_string']
sampleDoc = sample['func_documentation_string']

print(f"{sampleName}\n\n{sampleCode}\n\n{sampleDoc}")

In [ ]:
def formatFunction(example):
    instruction = "What does this function do?"
    input_text = example['func_code_string']
    output_text = example['docstring'] if 'docstring' in entry and entry['docstring'] else "No docstring provided."

    return {
        "instruction": instruction,
        "input": input_text,
        "output": output_text
    }

In [ ]:
dataset = dataset.map(formatFunction, batched=True,)